Notebook used to prepare and aggregate the international demand to be processed in the pipeline

In [1]:
import pandas as pd
import geopandas as gpd
import os
import re
import matplotlib.pyplot as plt
print(os.getcwd())
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())
pd.set_option('display.max_columns', None)

c:\Users\LMENENDEZ\GitHub\MultiModX\notebooks
C:\Users\LMENENDEZ\GitHub\MultiModX


# 1. Format and eliminate trips

In [2]:
# Trips during the week 22/09/2022 28/09/2022 (thursday to thursday)
# the day of study selected was Friday to put the air layer under pressure
all_trips = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|"
)

In [64]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
%autoreload
from script.trips_format import *

In [5]:
#associates each airport to the corresponding new NUTS
airports_to_NUTS={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_GMZ":("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")}

In [6]:
# we select trips that happened on friday
trips = all_trips[all_trips["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})
trips.head()

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300
2,20220923,P00,01009_AM,ES211,Álava,abroad_208,abroad,abroad,NaN,ground_Fra_08,NF,NF,abroad,D05_[50000-inf),01-02,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,FR,NaN,NaN,NaN,NaN,NaN,NaN,passenger,False,1.599,309.588,1.0,0.888333,0.399750,0.142133,0.071067,0.062183,0.035533,3,road-train-road,train_11213-train_11600,train_11213,train_11600,international_D,2,1,0,train_11213-train_11600,train_11208,train_11511
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200


In [7]:
trips=format_trips(trips,airports_to_NUTS)

17 columns were removed


In [8]:
# remove cercanías
trips=trips[~(((trips["origin"]=="ES424")&(trips["destination"]=="ES300"))|((trips["origin"]=="ES300")&(trips["destination"]=="ES424")))]

Now we remove trips with weird stations

In [9]:
# location of "ALL" train stops given by UiC
# However this list is still incomplete
stops_loc=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\rail_info\stops.txt").astype(str) # everything is a string here to match other formatting
stops_loc["stop_id"] = stops_loc["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [10]:
trips.loc[:,"weird_stations"] = trips["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))

In [11]:
unique_weird_stations=get_weird_stations(trips["weird_stations"])

In [12]:
MobA_stations_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\datos moba\train_stations\train_stations.shp")

In [13]:
# identifies all the un-localisable stations
nowhere_stations=set(unique_weird_stations)-set(MobA_stations_coord["ID"])
print(f"there are {len(nowhere_stations)} stations that are not in the data provided by MobA but appear in the trips dataframe")

there are 205 stations that are not in the data provided by MobA but appear in the trips dataframe


In [14]:
trips = trips[~trips["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]

Up to now we have removed:
- cercanías trips from Guadalajara to Madrid (and viceversa)
- trips with weird stations

# 2. Specifically format international trips

In [15]:
trips_abroad=trips[(trips["origin"]=="abroad")|(trips["destination"]=="abroad")]

In [16]:
# creates a dictionary of mcc code to country acronym
international_codes=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\countries mcc\mcc_to_nationality.txt", sep="|")
mcc_to_country=international_codes.set_index("mcc")["country"].to_dict()

In [17]:
trips_abroad=format_trips_abroad(trips_abroad,mcc_to_country)

We have eliminated all trips that leave Spain or come to Spain via ground and we have added the information about the origin/destination acronym

# 3. Recalibrate the international trips

In [18]:
coeffs_incoming=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\demand\incoming_trips_coefficients.csv")

In [19]:
coeffs_outgoing=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\demand\outgoing_trips_coefficients.csv")

In [20]:
#we have slightly different countries in coeffs_incoming and coeffs_outgoing
countries_incoming=set(coeffs_incoming["origin"])

countries_outgoing=set(coeffs_outgoing["destination"])

countries_selected=set.union(countries_outgoing,countries_incoming)-(countries_outgoing-countries_incoming)-(countries_incoming-countries_outgoing)

In [21]:
coeffs_incoming=coeffs_incoming[coeffs_incoming["origin"].isin(countries_selected)]
coeffs_outgoing=coeffs_outgoing[coeffs_outgoing["destination"].isin(countries_selected)]

In [22]:
trips_abroad.head()

,date,origin,origin_name,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,legs,nationality,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,trips
0,20220923,AD,AD,ES512,Girona,airport_GRO,NaN,NF,NF,P17*abroad_213*17233_AM*None*airport_GRO*00-01...,AD,0.374079,0.668807,0.430757,0.022671,0.056679,0.034007,2,plane-road,airport_GRO,airport_GRO,airport_GRO,international_O,1,0,1,airport_GRO,airport_GRO,airport_GRO,['air'],1.587
1,20220923,AD,AD,ES512,Girona,airport_GRO,NaN,NF,NF,P20*abroad_213*17233_AM*None*airport_GRO*00-01...,AD,0.374079,0.668807,0.430757,0.022671,0.056679,0.034007,2,plane-road,airport_GRO,airport_GRO,airport_GRO,international_O,1,0,1,airport_GRO,airport_GRO,airport_GRO,['air'],1.587
2,20220923,AD,AD,ES514,Tarragona,airport_REU,NaN,NF,NF,P18*abroad_213*4312310_AD*None*airport_REU*00-...,AD,2.709353,3.928561,1.964281,0.406403,0.067734,0.338669,2,plane-road,airport_REU,airport_REU,airport_REU,international_O,1,0,1,airport_REU,airport_REU,airport_REU,['air'],9.415
3,20220923,AD,AD,ES514,Tarragona,airport_REU,NaN,NF,NF,P23*abroad_213*4312310_AD*None*airport_REU*00-...,AD,0.456691,0.662201,0.331101,0.068504,0.011417,0.057086,2,plane-road,airport_REU,airport_REU,airport_REU,international_O,1,0,1,airport_REU,airport_REU,airport_REU,['air'],1.587
4,20220923,AD,AD,ES523,Valencia / València,airport_VLC,NaN,NF,NF,P13*abroad_213*4615903*None*airport_VLC*00-01*...,AD,4.090123,3.279816,1.813545,0.000000,0.115758,0.115758,2,plane-road,airport_VLC,airport_VLC,airport_VLC,international_O,1,0,1,airport_VLC,airport_VLC,airport_VLC,['air'],9.415


In [23]:
trips_abroad_selected=trips_abroad[(trips_abroad["origin"].isin(countries_selected))|(trips_abroad["destination"].isin(countries_selected))]

In [24]:
num_trips_abroad=trips_abroad["trips"].sum()
num_trips_abroad_selected=trips_abroad_selected["trips"].sum()
print(f"We have selected {num_trips_abroad_selected/num_trips_abroad*100:.2f}% of the original trips")

We have selected 89.41% of the original trips


In [25]:
trips_abroad_rescaled=rescale_trips(trips_abroad_selected,coeffs_incoming,coeffs_outgoing)

In [26]:
trips_abroad_rescaled.head()

,date,origin,origin_name,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,legs,nationality,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,trips
182,20220923,AT,AT,ES111,A Coruña,airport_LCG,NaN,NF,NF,P10*abroad_232*1503103*None*airport_LCG*00-01*...,AT,25.429960,19.375208,12.311330,2.220076,1.614601,1.412776,2,plane-road,airport_LCG,airport_LCG,airport_LCG,international_O,1,0,1,airport_LCG,airport_LCG,airport_LCG,['air'],62.363950
183,20220923,AT,AT,ES114,Pontevedra,airport_VGO,NaN,NF,NF,P19*abroad_232*3605707*None*airport_VGO*00-01*...,PL,4.672052,3.270437,1.635218,0.233603,0.467205,0.233603,2,plane-road,airport_VGO,airport_VGO,airport_VGO,international_O,1,0,1,airport_VGO,airport_VGO,airport_VGO,['air'],10.512118
184,20220923,AT,AT,ES120,Asturias,airport_VIT,NaN,NF,NF,P09*abroad_232*0105906*None*airport_VIT*00-01*...,AT,0.586863,0.365926,0.214032,0.069043,0.027617,0.041426,4,plane-road-bus-road,airport_VIT-bus_19-bus_2222,airport_VIT,bus_2222,international_O,2,0,1,airport_VIT-bus_19-bus_2222,airport_VIT,bus_2222,['air'],1.304907
185,20220923,AT,AT,ES120,Asturias,airport_VIT,NaN,NF,NF,P09*abroad_232*0105906*None*airport_VIT*00-01*...,AT,4.140809,2.581916,1.510177,0.487154,0.194862,0.292292,2,plane-road,airport_VIT,airport_VIT,airport_VIT,international_O,1,0,1,airport_VIT,airport_VIT,airport_VIT,['air'],9.207211
186,20220923,AT,AT,ES130,Cantabria,airport_SDR,NaN,NF,NF,P09*abroad_232*3901601*None*airport_SDR*00-01*...,PL,3.997566,4.589798,1.332522,0.148058,0.148058,0.296116,2,plane-road,airport_SDR,airport_SDR,airport_SDR,international_O,1,0,1,airport_SDR,airport_SDR,airport_SDR,['air'],10.512118


In [27]:
trips_abroad_rescaled=trips_abroad_rescaled.drop(["origin_purpose","destination_purpose","legs","nationality"],axis=1)

In [28]:
groupby_columns=["origin","origin_name","destination","destination_name","entry_point","exit_point","n_legs","node_sequence","mode_sequence","start_node","end_node","type","road_legs","train_legs","plane_legs","node_sequence_reduced","start_node_reduced","end_node_reduced","mode_tp","date"]
trips_abroad_rescaled=trips_abroad_rescaled.groupby(groupby_columns,as_index=False,dropna=False).sum()

In [29]:
airport_codes=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\airports_info\IATA_ICAO_Airport_codes_v1.3.csv")
iata_to_icao=airport_codes.set_index("IATA")["ICAO"].to_dict()

In [30]:
#change the entry_point or exit_point to icao code
trips_abroad_rescaled["entry_point"]=trips_abroad_rescaled["entry_point"].apply(lambda x: format_airports(x,iata_to_icao))
trips_abroad_rescaled["exit_point"]=trips_abroad_rescaled["exit_point"].apply(lambda x: format_airports(x,iata_to_icao))

# 4. Calculate the proportionality coefficients related to the airports

We have information about the international country of origin or destination of flights and about the national airport of origin or destination. Thus, the idea is to use the flight schedules to extract the proportion of international airports (per country) going to or coming from spain and apply this coefficient to our trips, to infer the airport proportionately

In [31]:
# download the flight schedules
flight_schedules=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\flights_schedules\flight_schedules_oag_es_v0.3.csv")

In [32]:
international_codes=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\countries mcc\mcc_to_nationality.txt", sep="|")

In [33]:
# build a dictionary country to acronym
country_to_acronym=international_codes.set_index("country_name")["country"].to_dict()
country_to_acronym["USA"]="US"
country_to_acronym["Russia"]="RU"

In [34]:
# the flights departing from each airport 
flights_capacity=flight_schedules.groupby(["origin","destination"])["seats"].sum().reset_index()

In [35]:
airport_codes["Country_acronym"]=airport_codes["Country"].map(country_to_acronym)

In [36]:
#dictionary of icao code to country acronym
airport_to_country=airport_codes.set_index("ICAO")["Country_acronym"].to_dict()

In [37]:
flights_capacity=map_and_aggregate_flight_capacity(flight_schedules,airport_to_country,selected_countries=countries_selected)

In [39]:
seats_from, seats_to = aggregate_seats_by_airport_and_country(flights_capacity)

We will divide the trips into incoming and outgoint trips. We start with the incoming trips

In [40]:
trips_abroad_incoming=trips_abroad_rescaled[trips_abroad_rescaled["destination"].str.startswith("ES")]
trips_abroad_outgoing=trips_abroad_rescaled[trips_abroad_rescaled["origin"].str.startswith("ES")]

In [41]:
trips_abroad_incoming["trips"].sum()

np.float64(258803.46661240142)

In [42]:
trips_abroad_incoming_final=assign_airport_incoming(trips_abroad_incoming,flights_capacity, seats_from)

In [43]:
trips_abroad_incoming_final.head()

,origin,origin_name,destination,destination_name,entry_point,exit_point,n_legs,node_sequence,mode_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,date,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,trips,seats,proportionality_coeff
6,LOWW,AT,ES213,Vizcaya,LEBB,NaN,2,airport_BIO,plane-road,airport_BIO,airport_BIO,international_O,1,0,1,airport_BIO,airport_BIO,airport_BIO,['air'],20220923,45.315458,22.190559,29.198104,1.635094,4.204527,1.868679,104.412420,180.0,1.0
8,LOWW,AT,ES243,Zaragoza,LEZG,NaN,2,airport_ZAZ,plane-road,airport_ZAZ,airport_ZAZ,international_O,1,0,1,airport_ZAZ,airport_ZAZ,airport_ZAZ,['air'],20220923,7.131027,1.106539,1.536859,0.000000,0.614744,0.122949,10.512118,291.0,1.0
9,LOWW,AT,ES300,Madrid,LEMD,NaN,2,airport_MAD,plane-road,airport_MAD,airport_MAD,international_O,1,0,1,airport_MAD,airport_MAD,airport_MAD,['air'],20220923,1053.144460,265.697160,316.329105,13.019643,59.311707,14.948479,1753.986908,831.0,1.0
10,LOWW,AT,ES300,Madrid,LEMD,NaN,4,airport_MAD-train_70101-train_17000,plane-road-train-road,airport_MAD,train_17000,international_O,2,1,1,airport_MAD,airport_MAD,airport_MAD,"['air', 'rail']",20220923,6.427342,1.621550,1.930557,0.079459,0.361979,0.091231,10.512118,831.0,1.0
11,LOWW,AT,ES300,Madrid,LEMD,NaN,4,airport_MAD-train_70102-train_18001,plane-road-train-road,airport_MAD,train_18001,international_O,2,1,1,airport_MAD,airport_MAD,airport_MAD,"['air', 'rail']",20220923,12.854684,3.243100,3.861114,0.158918,0.723959,0.182461,21.024235,831.0,1.0


In [44]:
trips_example=pd.merge(left=trips_abroad_incoming, left_on=["origin","entry_point"],right=flights_capacity,right_on=["country_origin","destination"],how="left")

In [45]:
num_trips_assigned=trips_example[trips_example["seats"].notna()]["trips"].sum()
num_trips=trips_example["trips"].sum()
print(f"There are {num_trips_assigned/num_trips*100:.2f}% of trips assigned an airport")


There are 99.62% of trips assigned an airport


Now we do the outgoing trips

In [46]:
trips_abroad_outgoing_final=assign_airport_outgoing(trips_abroad_outgoing,flights_capacity, seats_to)

In [47]:
print(trips_abroad_outgoing["trips"].sum())
print(trips_abroad_outgoing_final["trips"].sum())

243720.99312576538
232246.86922635167


In [48]:
trips_example=pd.merge(left=trips_abroad_outgoing, left_on=["destination","exit_point"],right=flights_capacity,right_on=["country_destination","origin"],how="left")

In [49]:
num_trips_assigned=trips_example[trips_example["seats"].notna()]["trips"].sum()
num_trips=trips_example["trips"].sum()
print(f"There are {num_trips_assigned/num_trips*100:.2f}% of trips assigned an airport")

There are 99.44% of trips assigned an airport


In [51]:
trips_abroad_outgoing_final.head()

,origin,origin_name,destination,destination_name,entry_point,exit_point,n_legs,node_sequence,mode_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,date,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,trips,seats,proportionality_coeff
1,ES111,A Coruña,LOWW,AT,NaN,LEMD,3,airport_SCQ-airport_MAD,road-plane-plane,airport_SCQ,airport_MAD,international_D,1,0,2,airport_SCQ-airport_MAD,airport_SCQ,airport_MAD,"['air', 'air']",20220923,22.283581,22.475680,3.841997,2.113098,1.344699,1.344699,53.403753,831.0,1.00000
4,ES111,A Coruña,EBBR,BE,NaN,LEMD,3,airport_LCG-airport_MAD,road-plane-plane,airport_LCG,airport_MAD,international_D,1,0,2,airport_LCG-airport_MAD,airport_LCG,airport_MAD,"['air', 'air']",20220923,6.627286,6.684418,1.142636,0.628450,0.399922,0.399922,15.882635,1858.0,0.90767
5,ES111,A Coruña,EBCI,BE,NaN,LEMD,3,airport_LCG-airport_MAD,road-plane-plane,airport_LCG,airport_MAD,international_D,1,0,2,airport_LCG-airport_MAD,airport_LCG,airport_MAD,"['air', 'air']",20220923,0.674143,0.679954,0.116231,0.063927,0.040681,0.040681,1.615618,189.0,0.09233
8,ES111,A Coruña,LSGG,CH,NaN,LEST,2,airport_SCQ,road-plane,airport_SCQ,airport_SCQ,international_D,1,0,1,airport_SCQ,airport_SCQ,airport_SCQ,['air'],20220923,114.718407,124.192747,22.957955,10.548303,6.181408,7.964878,302.020725,156.0,1.00000
10,ES111,A Coruña,EDDF,DE,NaN,LEST,2,airport_SCQ,road-plane,airport_SCQ,airport_SCQ,international_D,1,0,1,airport_SCQ,airport_SCQ,airport_SCQ,['air'],20220923,199.461600,201.181097,34.389931,18.914462,12.036476,12.036476,478.020041,150.0,1.00000


# 5. Group the demand so that it is ready to be sent

In [52]:
trips_abroad_final=pd.concat([trips_abroad_incoming_final, trips_abroad_outgoing_final], ignore_index=True)

In [70]:
combined_df=trips_format_to_pipeline(trips_abroad_final)

In [69]:
trips_intra=pd.read_csv(r"C:\Users\LMENENDEZ\GitHub\MultiModX\demand_ES_MD_intra_v0.4.csv")

In [71]:
trips_international=pd.concat([trips_intra,combined_df],ignore_index=True)

In [74]:
trips_international.to_csv("demand_ES_MD_international_v0.1.csv",index=False)

In [ ]:
trips_abroad_selected["trips_abroad"]

For some archetypes we get 0 trips. Should we remove those 0's?